# Sesiunea 10 – Algoritmi Avansați și Optimizare
_Notebook de exerciții (fără soluții)._

## Exercițiul 1 — Sortează 100.000 de numere cu Merge Sort și măsoară timpul
- Generează o listă cu **100.000** de numere aleatoare.
- Implementează **Merge Sort** (divide et impera).
- Măsoară timpul de execuție folosind `time.time()` și afișează-l.
- Verifică că lista rezultată este sortată corect.
- Notează complexitatea teoretică: `O(n log n)` și discută diferențele observate față de timp.

In [31]:
import random, time

# numbers = [random.randint(-50000, 50000) for _ in range(0, 99_999)]
numbers = [random.randint(-100_000, 100_000) for _ in range(0, 100_000)]

def selection_sort(a):
    for i in range(len(a)):
        for y in range(i, len(a)):
            if a[y] < a[i]:
                a[i], a[y] = a[y], a[i]

def insertion_sort(a):
    for i in range(1, len(a)):
        for j in range(i, 0, -1):
            if a[j] < a[j - 1]:
                a[j], a[j - 1] = a[j - 1], a[j]
            else:
                break

def merge_two(a: list, b: list):
    i = j = 0
    new_full = []

    while i < len(a) and j < len(b):
        if a[i] < b[j]:
            new_full.append(a[i])
            i += 1
        else:
            new_full.append(b[j])
            j += 1

    new_full.extend(a[i:])
    new_full.extend(b[j:])

    return new_full


def merge_sort(a):
    if len(a) <= 4:
        insertion_sort(a)
        return

    mid = len(a) // 2
    left = a[:mid]
    right = a[mid:]

    merge_sort(left)
    merge_sort(right)
    
    a[:] = merge_two(left, right)


# print(numbers)

start = time.time()
merge_sort(numbers)
end = time.time()

# print(numbers)
print("merge time taken:", round(end - start, 3), "s", "| python overhead + lack of extreme optimizations in algo")

numbers = [random.randint(-100_000, 100_000) for _ in range(0, 100_000)]

start = time.time()
numbers.sort()
end = time.time()

print("timsort c implement time taken:", round(end - start, 3), "s")

merge time taken: 0.28 s | python overhead + lack of extreme optimizations in algo
timsort c implement time taken: 0.017 s


## Exercițiul 2 — Recursivitate & Memoizare: compară factorial cu și fără memo
- Implementează `factorial(n)` recursiv **fără** memoizare.
- Implementează o variantă **cu memoizare** (cache).
- Cronometrează apelurile pentru mai multe valori (ex.: `n = 2000, 4000, 6000` sau o plajă sigură pentru mediul tău).
- Compară timpii și discută impactul memoizării asupra performanței și consumului de memorie.
- (Opțional) Repetă experimentul pentru altă funcție recursivă costisitoare (ex.: Fibonacci).

In [32]:
import time
import sys
sys.setrecursionlimit(10000)


def factorial(n):
    global fac_memo
    if n in fac_memo:
        return fac_memo[n]
    last = 1
    if n > 1:
        last = factorial(n - 1)
        fac_memo[n] = n * last

    return fac_memo[n]


def fib(n): 
    if n <= 1: 
        return n 
    return fib(n-1) + fib(n-2)


def fib_cache(n):
    if n in fib_memo:
        return fib_memo[n]
    
    fib_memo[n] = fib_cache(n-1) + fib_cache(n-2)

    return fib_memo[n]


def test_sm(title, funct, count=2000, memo=None):
    print(f"\n{title}")
    print("=" * len(title))

    if memo:
        print(f"Initial mem size : {sys.getsizeof(memo) // 1024:>8} KB")

    start = time.time()
    funct(count)
    mid = time.time()
    funct(count)
    end = time.time()

    if memo:
        print(f"Final mem size   : {sys.getsizeof(memo) // 1024:>8} KB")

    print(f"First run time   : {(mid - start) * 1000:>8.3f} ms")
    print(f"Second run time  : {(end - mid) * 1000:>8.3f} ms")


fac_memo = {1: 1}
test_sm("Factorial", factorial, 8000, fac_memo)

fib_memo = {0: 0, 1: 1}
test_sm("Fibonacci Cached", fib_cache, 2000, fib_memo)

test_sm("Fibonacci Plain", fib, 30)


Factorial
Initial mem size :        0 KB
Final mem size   :      288 KB
First run time   :   15.040 ms
Second run time  :    0.176 ms

Fibonacci Cached
Initial mem size :        0 KB
Final mem size   :       72 KB
First run time   :    0.834 ms
Second run time  :    0.000 ms

Fibonacci Plain
First run time   :  126.131 ms
Second run time  :  121.461 ms


## Exercițiul 3 — Greedy: Interval Scheduling (maximizarea participărilor)
- Primești o listă de intervale `[(start, end), ...]` care reprezintă evenimente într-o zi.
- Alege un **subansamblu maxim** de intervale **ne-suprapuse** pentru a maximiza numărul de participări.
- Strategie clasică greedy: **sortează după `end` crescător** și selectează iterativ.
- Afișează atât numărul de evenimente selectate, cât și lista intervalelor alese.

In [33]:
events = [
    ("08:00", "09:30"),  # Morning meeting
    ("09:15", "10:00"),  # Overlaps with above
    ("10:30", "11:30"),
    ("11:00", "12:00"),  # Overlaps with above
    ("13:00", "14:00"),
    ("14:00", "15:00"),
    ("15:30", "17:00"),
    ("16:45", "18:00"),  # Overlaps with above
    ("19:00", "20:00")
]


def time_to_minutes(t: str) -> int:
    h, m = map(int, t.split(":"))
    return h * 60 + m


def minutes_to_time(m: int) -> str:
    h = m // 60
    m = m % 60
    return f"{h:02d}:{m:02d}"


def greedy(events: list):
    selected = []
    current_end = 0

    for start, end in events:
        if start >= current_end:
            selected.append((start, end))
            current_end = end

    return selected


events.sort(key=lambda x: time_to_minutes(x[1]))
print(events)
new_events = [(time_to_minutes(x[0]), time_to_minutes(x[1])) for x in events] # convert to minutes pairs

selected = greedy(new_events)
re_made = [(minutes_to_time(s), minutes_to_time(e)) for s, e in selected]
print(re_made)


[('08:00', '09:30'), ('09:15', '10:00'), ('10:30', '11:30'), ('11:00', '12:00'), ('13:00', '14:00'), ('14:00', '15:00'), ('15:30', '17:00'), ('16:45', '18:00'), ('19:00', '20:00')]
[('08:00', '09:30'), ('10:30', '11:30'), ('13:00', '14:00'), ('14:00', '15:00'), ('15:30', '17:00'), ('19:00', '20:00')]


## Exercițiul 4 — Sortează un fișier CSV cu Merge Sort
- Încarcă un fișier CSV care conține o coloană numerică (poți genera un fișier de test).
- Extrage coloana numerică într-o listă și sorteaz-o cu **Merge Sort** implementat de tine.
- Scrie rezultatul într-un nou fișier CSV și validează ordinea.
- Cronometrează timpii de citire, sortare și scriere.

In [ ]:
import csv
import time
import random

FILE_READ = "input.csv"
FILE_WRITE = "output.csv"


# Generează fișier CSV
def generate_csv(filename=FILE_READ, n=100):
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "value"])
        for i in range(n):
            writer.writerow([i, random.randint(0, 1000)])


def values_extract(filename=FILE_READ):
    final = []
    with open(filename, "r", newline="") as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            final.append(int(row[1]))

    return final
            

start_gen = time.time()
generate_csv()
print("Gen:".ljust(12), f"{time.time() - start_gen:.6f}s")

start_read = time.time()
values = values_extract()
print("Read:".ljust(12), f"{time.time() - start_read:.6f}s")

start_sort = time.time()
merge_sort(values)
print("Sort:".ljust(12), f"{time.time() - start_sort:.6f}s")

start_validate = time.time()
for i in range(len(values) - 1):
    if values[i] > values[i + 1]:
        raise ValueError(f"Invalid Order [{i}]{values[i]} > [{i + 1}]{values[i + 1]}")
print("Validate:".ljust(12), f"{time.time() - start_validate:.6f}s")

start_write = time.time()
with open(FILE_WRITE, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "value"])
    for id, v in enumerate(values):
        writer.writerow([id, v])
print("Write:".ljust(12), f"{time.time() - start_write:.6f}s")


Gen: 0.001032s
Read: 0.000878s
Sort: 0.000353s
Validate: 0.000212s
Write: 0.000555s


## Exercițiul 5 — Backtracking: permutările unui cuvânt
- Primește un cuvânt (ex.: `"level"`).
- Generează **toate permutările** sale folosind **backtracking**.
- Elimină duplicatele atunci când apar litere repetate.
- Afișează numărul total de permutări unice și câteva exemple.

## Exercițiul 6 — Greedy: planifică activități într-un buget de timp
- Ai activități cu durate `d_i` și o limită totală de timp `T` într-o zi.
- Folosește o strategie greedy (ex.: **SJF — Shortest Job First**) pentru a **maximiza numărul** de activități finalizate în `T`.
- Afișează setul selectat, timpul total folosit și numărul de activități realizate.

## Exercițiul 7 — Backtracking: problema celor 8 regine (afișare ASCII)
- Plasează **8 regine** pe o tablă de șah 8×8 astfel încât **nicio** pereche să nu se atace.
- Găsește **toate soluțiile** folosind backtracking.
- Afișează cel puțin o soluție în **ASCII**, de exemplu cu `Q` pentru regină și `.` pentru gol.
- (Opțional) Permite parametrizarea pentru `n` regine.